In [1]:
import pandas as pd 
import numpy as np 
import os 
from pathlib import Path 

In [19]:
## Add team metrics to regular season games to create dataset for supervised ML model
pd.set_option("display.max_columns",None)

stats_summary = pd.read_csv('../../data/preprocessing/womens_summary_season_data.csv')
womens_reg_season_data = pd.read_csv('../../data/WRegularSeasonCompactResults.csv')
womens_reg_season_data = womens_reg_season_data[womens_reg_season_data['Season'] >= 2010]


# Duplicate data to there is a record for each losing and winning team 
womens_reg_season_data['Team1'] = womens_reg_season_data['WTeamID']
womens_reg_season_data['Team2'] = womens_reg_season_data['LTeamID']
womens_reg_season_data['Team1_Wins'] = 1 

flipped = womens_reg_season_data.copy() 
flipped['Team1'], flipped['Team2'] = flipped['Team2'], flipped['Team1'] 
flipped['Team1_Wins'] = 0 

games = pd.concat([womens_reg_season_data, flipped])
games.drop(columns=['WTeamID','WScore','LTeamID','LScore','WLoc','NumOT'], inplace=True)

# Merge games dataset with teams stats 
full_games = games.merge(stats_summary, left_on=['Season','Team1'], right_on=['Season', 'TeamID'], how='left')
full_games = full_games.rename(columns={col: col + "_1" for col in stats_summary.columns if col not in ["Season", "TeamID"]})

full_games = full_games.merge(stats_summary, left_on=['Season','Team2'], right_on=['Season', 'TeamID'], how='left')
full_games = full_games.rename(columns={col: col + "_2" for col in stats_summary.columns if col not in ["Season", "TeamID"]})
full_games.drop(columns=['TeamID_x', 'TeamName_1', 'TeamID_y', 'TeamName_2'], inplace=True)

full_games


,Season,DayNum,Team1,Team2,Team1_Wins,Win_Percentage_1,Points_Per_Game_1,FG_Percentage_1,Threes_Per_Game_1,Three_Point_Percentage_1,Free_Throws_Per_Game_1,Free_Throw_Percentage_1,Offensive_Rebound_Rate_1,Defensive_Rebound_Rate_1,Turnovers_Per_Game_1,Opp_FG_Percentage_1,Opp_Three_Point_Percentage_1,Opp_Free_Throws_Per_Game_1,Opp_Turnovers_Per_Game_1,Win_pct_last_10_games_1,Win_Percentage_2,Points_Per_Game_2,FG_Percentage_2,Threes_Per_Game_2,Three_Point_Percentage_2,Free_Throws_Per_Game_2,Free_Throw_Percentage_2,Offensive_Rebound_Rate_2,Defensive_Rebound_Rate_2,Turnovers_Per_Game_2,Opp_FG_Percentage_2,Opp_Three_Point_Percentage_2,Opp_Free_Throws_Per_Game_2,Opp_Turnovers_Per_Game_2,Win_pct_last_10_games_2
0,2010,11,3103,3237,1,0.566667,62.233333,40.229192,12.466667,31.550802,19.600000,70.578231,39.664804,63.351254,18.266667,38.144330,31.868132,18.833333,18.700000,0.6,0.071429,51.321429,36.697248,11.964286,22.686567,17.785714,64.457831,33.930348,62.327416,23.571429,42.803504,37.111111,21.392857,16.785714,0.1
1,2010,11,3104,3399,1,0.379310,64.137931,39.811216,14.724138,29.508197,16.793103,61.601643,35.353535,63.567202,19.172414,39.736239,30.897704,21.965517,19.241379,0.3,0.448276,65.241379,38.811189,19.482759,31.327434,20.827586,63.410596,34.096692,67.625899,16.724138,40.781441,32.832080,25.206897,15.793103,0.6
2,2010,11,3110,3224,1,0.619048,60.428571,40.017668,15.285714,30.529595,16.809524,75.070822,37.174211,65.369128,15.190476,37.803780,28.859060,15.000000,16.428571,0.8,0.517241,56.862069,34.720571,11.413793,24.169184,19.862069,69.618056,38.799662,65.033622,15.758621,39.041534,30.297030,17.482759,17.034483,0.6
3,2010,11,3111,3267,1,0.612903,66.419355,43.329718,12.387097,34.635417,16.612903,63.689320,39.198606,67.034700,21.225806,36.348774,32.871972,20.548387,18.161290,0.7,0.448276,59.310345,40.371926,9.275862,28.624535,18.172414,56.356736,38.584071,65.796831,17.448276,39.742805,38.274336,17.448276,16.310345,0.2
4,2010,11,3119,3447,1,0.333333,52.291667,35.797665,16.916667,27.339901,13.875000,67.267267,33.107345,62.500000,17.208333,37.899543,34.493671,18.458333,16.291667,0.3,0.300000,60.333333,37.290034,16.566667,31.388330,15.266667,70.305677,35.672515,64.305428,19.766667,40.988547,36.363636,22.066667,19.500000,0.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161689,2025,106,3416,3242,0,0.360000,67.080000,41.105929,14.160000,30.790960,18.920000,70.613108,32.680539,72.081218,15.960000,43.116185,34.523810,15.960000,15.760000,0.2,0.576923,66.769231,42.659884,18.807692,35.173824,20.038462,74.856046,19.633508,73.072215,13.807692,42.024129,34.352518,16.346154,14.115385,0.4
161690,2025,106,3428,3329,0,0.769231,77.769231,47.572178,26.230769,37.976540,16.000000,75.240385,30.298913,78.239609,15.192308,40.498652,32.842105,12.961538,16.461538,0.7,0.807692,78.038462,46.230530,23.653846,35.121951,18.153846,69.703390,32.957393,77.324263,14.692308,36.274510,26.824818,15.153846,16.000000,0.7
161691,2025,106,3194,3349,0,0.370370,60.740741,36.303030,20.851852,31.261101,16.111111,61.149425,29.607251,69.005236,15.629630,40.257461,32.730263,12.407407,15.111111,0.1,0.500000,66.384615,41.176471,20.153846,31.679389,13.961538,71.074380,28.486293,76.553980,15.423077,38.695090,30.212766,16.846154,13.846154,0.4
161692,2025,106,3150,3378,0,0.320000,60.760000,37.799718,17.600000,27.500000,18.520000,70.410367,32.922318,67.185473,16.480000,41.742797,31.746032,20.320000,15.880000,0.3,0.703704,66.185185,42.111801,15.592593,34.441805,15.148148,69.926650,32.429379,73.155985,13.296296,40.725806,31.263383,14.592593,15.814815,0.8


In [21]:
## Add team metrics to regular season games to create dataset for supervised ML model
pd.set_option("display.max_columns",None)

womens_tourney_data = pd.read_csv('../../data/WNCAATourneyCompactResults.csv')
womens_tourney_data = womens_tourney_data[womens_tourney_data['Season'] >= 2010]

womens_tourney_seeds = pd.read_csv("../../data/WNCAATourneySeeds.csv")
womens_tourney_seeds['Seed'] = womens_tourney_seeds['Seed'].str[1:].str.rstrip('ab')

# Duplicate data to there is a record for each losing and winning team 
womens_tourney_data['Team1'] = womens_tourney_data['WTeamID']
womens_tourney_data['Team2'] = womens_tourney_data['LTeamID']
womens_tourney_data['Team1_Wins'] = 1 

flipped = womens_tourney_data.copy() 
flipped['Team1'], flipped['Team2'] = flipped['Team2'], flipped['Team1'] 
flipped['Team1_Wins'] = 0 

games = pd.concat([womens_tourney_data, flipped])
games.drop(columns=['WTeamID','WScore','LTeamID','LScore','WLoc','NumOT'], inplace=True)

# Merge games dataset with teams stats 
tourney_games = games.merge(stats_summary, left_on=['Season','Team1'], right_on=['Season', 'TeamID'], how='left')
tourney_games = tourney_games.rename(columns={col: col + "_1" for col in stats_summary.columns if col not in ["Season", "TeamID"]})

tourney_games = tourney_games.merge(stats_summary, left_on=['Season','Team2'], right_on=['Season', 'TeamID'], how='left')
tourney_games = tourney_games.rename(columns={col: col + "_2" for col in stats_summary.columns if col not in ["Season", "TeamID"]})
tourney_games.drop(columns=['TeamID_x', 'TeamName_1', 'TeamID_y', 'TeamName_2'], inplace=True)

tourney_games = tourney_games.merge(womens_tourney_seeds, how='left', left_on=['Season','Team1'], right_on=['Season', 'TeamID'])
tourney_games = tourney_games.rename(columns={'Seed':'Seed_1'})
tourney_games = tourney_games.merge(womens_tourney_seeds, how='left', left_on=['Season','Team2'], right_on=['Season', 'TeamID'])
tourney_games = tourney_games.rename(columns={'Seed':'Seed_2'})
tourney_games = tourney_games.drop(columns=['TeamID_x', 'TeamID_y'])
tourney_games


,Season,DayNum,Team1,Team2,Team1_Wins,Win_Percentage_1,Points_Per_Game_1,FG_Percentage_1,Threes_Per_Game_1,Three_Point_Percentage_1,Free_Throws_Per_Game_1,Free_Throw_Percentage_1,Offensive_Rebound_Rate_1,Defensive_Rebound_Rate_1,Turnovers_Per_Game_1,Opp_FG_Percentage_1,Opp_Three_Point_Percentage_1,Opp_Free_Throws_Per_Game_1,Opp_Turnovers_Per_Game_1,Win_pct_last_10_games_1,Win_Percentage_2,Points_Per_Game_2,FG_Percentage_2,Threes_Per_Game_2,Three_Point_Percentage_2,Free_Throws_Per_Game_2,Free_Throw_Percentage_2,Offensive_Rebound_Rate_2,Defensive_Rebound_Rate_2,Turnovers_Per_Game_2,Opp_FG_Percentage_2,Opp_Three_Point_Percentage_2,Opp_Free_Throws_Per_Game_2,Opp_Turnovers_Per_Game_2,Win_pct_last_10_games_2,Seed_1,Seed_2
0,2010,138,3124,3201,1,0.718750,72.093750,46.311703,8.656250,28.519856,24.000000,72.656250,35.878565,71.000000,16.875000,33.732535,27.742947,13.187500,15.718750,0.6,0.818182,73.454545,41.978346,24.848485,35.487805,18.242424,70.930233,35.865309,66.748971,15.636364,40.459647,29.978118,16.303030,19.878788,0.9,04,13
1,2010,138,3173,3395,1,0.807692,70.769231,42.062500,17.461538,31.277533,19.346154,69.980119,38.261738,69.923372,16.961538,36.762779,32.191781,16.576923,18.346154,0.8,0.733333,72.300000,41.488223,19.200000,35.763889,18.400000,74.818841,34.763948,63.472563,16.033333,36.571748,28.429752,16.300000,20.666667,0.7,08,09
2,2010,138,3181,3214,1,0.843750,71.593750,42.107843,14.468750,31.533477,20.156250,66.201550,44.645669,67.186218,17.906250,35.160350,32.015066,18.125000,23.218750,0.9,0.633333,62.833333,36.938436,16.100000,30.434783,18.933333,71.478873,36.415247,64.211520,15.533333,38.050734,27.741935,22.000000,22.133333,0.9,02,15
3,2010,138,3199,3256,1,0.833333,74.466667,44.594595,16.100000,37.888199,21.800000,71.406728,40.487348,67.049180,19.033333,35.814214,28.752260,17.466667,19.200000,0.8,0.741935,74.258065,43.446852,12.451613,31.606218,23.354839,68.508287,37.715694,68.448404,17.096774,37.311178,27.643785,18.000000,17.451613,0.8,03,14
4,2010,138,3207,3265,1,0.800000,68.033333,39.546460,20.066667,31.561462,19.533333,71.843003,40.775862,61.668427,16.300000,40.381992,28.672566,18.766667,24.600000,0.7,0.787879,68.181818,42.056583,19.030303,34.394904,18.484848,80.000000,31.037567,64.727815,13.484848,34.910026,29.645390,14.909091,15.727273,0.8,05,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1783,2024,147,3425,3163,0,0.838710,74.677419,42.568617,21.225806,35.562310,18.838710,74.828767,31.795878,75.794872,11.806452,38.790233,30.671506,15.387097,14.548387,0.9,0.848485,79.878788,49.557522,19.787879,35.834609,15.484848,75.538160,29.257642,77.150786,12.363636,36.231130,28.370787,12.242424,15.969697,0.9,01,03
1784,2024,147,3261,3234,0,0.848485,86.696970,46.712018,13.212121,32.110092,26.727273,74.943311,44.225604,75.893599,15.212121,36.731707,29.613734,15.969697,18.939394,0.9,0.878788,92.848485,50.322284,29.606061,38.178096,19.575758,78.173375,29.596853,76.206051,13.424242,40.101056,31.055156,14.969697,14.424242,0.8,03,01
1785,2024,151,3163,3234,0,0.848485,79.878788,49.557522,19.787879,35.834609,15.484848,75.538160,29.257642,77.150786,12.363636,36.231130,28.370787,12.242424,15.969697,0.9,0.878788,92.848485,50.322284,29.606061,38.178096,19.575758,78.173375,29.596853,76.206051,13.424242,40.101056,31.055156,14.969697,14.424242,0.8,03,01
1786,2024,151,3301,3376,0,0.818182,73.818182,42.919075,18.636364,32.357724,18.696970,73.743922,28.028933,77.186010,12.333333,35.453695,29.178082,10.060606,12.696970,0.7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,03,01


In [ ]:
output_dir = Path("../..") / "data" / "modeling"
output_dir.mkdir(parents=True, exist_ok =True)
season_output_path = output_dir / "womens_reg_season_ml.csv"
tourney_output_path = output_dir / "womens_tourney_ml.csv"
full_games.to_csv(season_output_path, index=False)
tourney_games.to_csv(tourney_output_path, index=False)